# General information

Мы узнали несколько важных вещей: 

* как можно построить работающую рекомендательную систему;
* как рекомендательные системы используются в продакшне;
* как замерять их влияние на бизнес. 

Пора применить эти знания в своем собственном проекте!

Датасет

У нас будет история оценок пользователя вместе с его обзором. Мы можем использовать текст рецензии в качестве дополнительной информации. Все оценки пользователей нормированы для бинарной классификации: если человек поставил оценку продукту больше 3 (не включительно), то мы считаем, что продукт ему понравился, если меньше 4, то продукт не понравился.

test.csv - набор данных, для которого необходимо сделать предсказания. У каждого набора userid, itemid есть свой id, для которого вы должны сделать предсказание.

* overall - рейтинг, который поставил пользователь;
* verified - был ли отзыв верифицирован;
* reviewTime - когда был написан отзыв;
* reviewerName - имя пользователя;
* reviewText - текст отзыва;
* summary - сжатый отзыв;
* unixReviewTime - дата отзыва в формате unix;
* vote - количество голосований за отзыв;
* style - метаданные;
* image - изображение продукта;
* userid - id пользователя;
* itemid - id товара;
* id - id для предсказания.

# 1. IMPORT, FUNCTIONS, SETUP

In [1]:
# !pip install nmslib

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# Any results you write to the current directory are saved as output.
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
# Импорт библиотек
import numpy as np
import pandas as pd
import scipy.sparse as sparse
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sklearn
import json
import re

import pandas_profiling

from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
from pandas import Series
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import f_regression, mutual_info_regression
from scipy.stats import ttest_ind
from itertools import combinations
from collections import Counter
# import nmslib

from sklearn.metrics import auc, roc_auc_score, roc_curve
import warnings 
warnings.simplefilter('ignore')

scaler = MinMaxScaler()
cnt = Counter()

%matplotlib inline
sns.set()

## Functions

В данном блоке собраны пользовательские функции.

In [4]:
def intitial_eda_checks(df, missing_percent):
    '''
    Функция принимает на вход датафрейм, а также заданный порог % пустых значений, который хотим обработать. 
    На выход выводит на экран информацию о сумме пустых значений для всех колонок, а также проце
    '''
    if df.isnull().sum().sum() > 0:
        mask_total = df.isnull().sum().sort_values(ascending=False)
        total = mask_total[mask_total > 0]

        mask_percent = df.isnull().mean().sort_values(ascending=False)
        percent = mask_percent[mask_percent > 0]

        series = mask_percent[mask_percent > missing_percent]
        columns = series.index.to_list()

        missing_data = pd.DataFrame(pd.concat(
            [total, round(percent*100, 2)], axis=1, keys=['Количество', '%']))
        print('Сумма и процент значений NaN:\n \n')
        display(missing_data)
    else:
        print('NaN значения не найдены.')

In [5]:
def iqr_analysis(series, mode=False):
    """
    Функция выводит инфорамцию о границах выборосов для признака.
    Если mode = True, возвращается верхняя и нижняя границы выбросов. Иначе, просто выводится информация на экран.
    """
    IQR = series.quantile(0.75) - series.quantile(0.25)
    perc25 = series.quantile(0.25)
    perc75 = series.quantile(0.75)

    f = perc25 - 1.5*IQR
    l = perc75 + 1.5*IQR

    if mode:
        return f, l

    print(
        "\n25-й перцентиль: {},".format(perc25),
        "\n75-й перцентиль: {},".format(perc75),
        "\nIQR: {}, ".format(IQR),
        "\nГраницы выбросов: [{f}, {l}].".format(
            f=perc25 - 1.5*IQR, l=perc75 + 1.5*IQR),
        "\n\nМинимальное значение признака: {}.".format(series.min()),
        "\nМаксимальное значение признака: {} .\n".format(series.max()))

    if series.min() < f:
        print("Найдены выбросы по нижней границе признака! Количество: {}, {}%".format(series.where(
            series < f).count(), round(series.where(series < f).count()/series.count()*100, 2)))
    if series.max() > l:
        print("Найдены выбросы по верхней границе признака! Количество: {}, {}%".format(series.where(
            series > l).count(), round(series.where(series > l).count()/series.count()*100, 2)))

In [6]:
def get_stat_dif(column):
    """ 
    Поиск статистически значимых различий для колонки с помощью теста Стьюдента.
    """
    cols = data.loc[:, column].value_counts().index[:]
    combinations_all = list(combinations(cols, 2))

    tmp = data[data['train'] == 1]

    for comb in combinations_all:
        if ttest_ind(tmp.loc[data[data['train'] == 1].loc[:, column] == comb[0], 'price'],
                     tmp.loc[data[data['train'] == 1].loc[:, column] == comb[1], 'price']).pvalue <= 0.05/len(combinations_all):  # учли поправку Бонферони
            # print('Найдены статистически значимые различия для колонки и комбинаций', column, comb)
            pass
        else:
            print(
                'Не найдены статистически значимые различия для колонки и комбинации', column, comb)
            return column
            break

In [7]:
# Пропишем функцию для отображения ROC-кривой:
def roc_auc_curve(y_true, y_pred_prob):
    fpr, tpr, _ = roc_curve(y_true, y_pred_prob)
    plt.figure()
    plt.plot([0, 1], label='Случайный классификатор', linestyle='--')
    plt.plot(fpr, tpr, label = 'LightFM')
    plt.title('ROC AUC = %0.3f' % roc_auc_score(y_true, y_pred_prob))
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.legend(loc = 'lower right')
    plt.show()

In [8]:
# Пропишем функцию для создания категории в зависимости от даты написания отзыва
def cat_date(x):
    if x <= ts25: x = 'old'
    elif ts25 < x <= ts50: x = 'middle_old'
    elif ts50 < x <= ts75: x = 'middle_new'
    elif ts75 < x: x = 'new'
    return x   

In [9]:
# Пропишем функцию для создания ранга (категоризация признака)
def cat_rank(x):
    if x < 1093: x = 'high'
    elif 1093 <= x <= 18000: x = 'middle'
    elif 18000 < x: x = 'low'
    return x  

In [10]:
# # Вспомогательная функция для поиска по графу
# def nearest_item_nms(itemid, index, n=10):
#     nn = index.knnQuery(item_embeddings[itemid], k=n)
#     return nn

## Setup

In [11]:
# Задание условий
LEARNING_RATE = 0.09
RANDOM_STATE = 32
NUM_THREADS = 4
NUM_COMPONENTS = 160 
NUM_EPOCHS = 20
LEARNING_SCHEDULE = 'adagrad'
LOSS_FUNCTION = 'logistic'

!pip freeze > requirements.txt

# 2. DATA AND PRELIMINARY ANALYSIS

## Data Storage

In [12]:
!ls '../input'

In [13]:
# Importing datasets.
train = pd.read_csv('/kaggle/input/recommendationsv4/train.csv')
test = pd.read_csv('/kaggle/input/recommendationsv4/test.csv')
submission = pd.read_csv('/kaggle/input/recommendationsv4/sample_submission.csv')

with open('/kaggle/input/recommendationsv4/meta_Grocery_and_Gourmet_Food.json') as data:
    meta_list = []
    for line in data.readlines():
        meta_list.append(json.loads(line))
        
meta = pd.DataFrame(meta_list)

In [14]:
# Checking the data.
test.info()
test.head()

In [15]:
# Checking the data.
train.info()
train.head()

In [16]:
# Checking the data.
meta.info()
meta.head()

Первичный анализ трех датасетов, говорит, что это достаточно большие датасеты. Признаки из тренировочно и мета датасеты не полностью сопадают с тестовым. В тренировочном датасете на четыре столбца больше:

* overall - оценка по пятибальной шкале
* reviewText - текст отзыва
* summary - краткое содержание отзыва
* rating - целевая переменная,

отсутствует Id - id для предсказания.

# 3. EDA AND BASIC DATA CLEANING

Проведем анализ данных, сделаем базовый препроцессинг данных.

## 3.1 Дубликаты

In [17]:
# # В целях эксперимента Закомментируем строки
# # Удалим дубликаты из данных train
# train = train.drop_duplicates()

In [18]:
# # В целях эксперимента Закомментируем строки
# # Удалим дубликаты из данных meta
# meta = meta.iloc[meta.astype(str).drop_duplicates().index]

Закомментировано, т.к. удаление дубликатов привело к снижению значения целевой метрики.

## 3.2 Унификация признаков в test, train, meta и объединение

Предварительный анализ test данных.

Посмотрим на данные в test, которые нужно предсказывать, на особенности данных.

In [19]:
#ProfileReport(test, title="Pandas Profiling Report for Test Dataset")

Результаты предварительного EDA:

* в данных много дублей;
* в колонках revierName, reviewText, vote, style, image много пропусков;

Объединение датасетов возможно осуществить по признаку 'asin', который является уникальным идентификатором товара на Amazon.

In [20]:
# Для анализа склеиваем все датафреймы по общим колонкам, добавляем признак train
test['train'] = 0  # помечаем где у нас test

# В тесте у нас нет значения rating, поэтому пока просто заполняем нулями
test['rating'] = 0

df = test.append(train, sort=False).reset_index(drop=True)
data = pd.merge(df, meta, on='asin')

# # # В целях эксперимента Создадим отдельный train датафрейм
# train_sub = pd.merge(train, meta, on='asin')

In [21]:
# Посмотрим, что получилось
data.info()

## 3.3 Обработка пропусков

Посмотрим в целом на наличие пустых значений и определим стратегию работы с ними.

In [22]:
# Запускаем функцию вывода всех пустых значений
print("Пропуски для DATA датафрейа.\n")
intitial_eda_checks(data, 0)

Выводы и стратегия обработки:

В 23 столбцах присутствуют пропуски.

* image_y - заполним нулями пропущенные значения;
* rank - поисследовать пропуски и подумать над способами заполнения;
* main_cat - поисследовать пропуски и подумать над способами заполнения.

Остальные признаки рассмотрим в ходе детального анализа.


### image_y

In [23]:
# Заполним нулями пропущенные значения
data['image_y'] = data['image_y'].fillna(0)

### main_cat

In [24]:
# Заполним пропущенные значения
data['main_cat'] = data['main_cat'].fillna('no_category_provided')

### train

In [25]:
# Заполним пропущенные значения для train датасета
data['train'] = data['train'].fillna(1)

## 3.4 Очистка данных

Проведем чистку и дополнительную подготовку данных.

### Verified Purchase

In [26]:
# Преобразуем значения в колонке verified
verified_dict = {
    True: 1,
    False: 0
}

data['verified'] = data['verified'].map(verified_dict)

### Review Time

В описании сказано, что это время написания отзыва; у нас также имеется признак в юниксовом формате unixReviewTime. Очевидно, что признак reviewTime можно и нужно удалить, так как это стопроцентная корреляция.

In [27]:
# Создадим список, в который будем добавлять колонки-кандидаты на удаление из датасета
cols_removal = ['reviewTime']

### ASIN (Amazon Standard Identification Number)

In [28]:
# Рассмотрим данные в столбце
print(data.asin.describe())

Уникальных товаров в нашей базе данных 41320. Непонятно, как признак может быть полезен. Удалим столбец.

In [29]:
# Добавим колонку в список на удаление из датасета
cols_removal.append('asin')

### Reviewer Name

In [30]:
# Рассмотрим данные в столбце
data['reviewerName'].value_counts().head(10)

In [31]:
# Создадим новый признак
data['kindle_customer'] = 0

counter = 0

for name in data['reviewerName']:     
    if name == 'Kindle Customer':
        data.at[counter,'kindle_customer'] = 1
        counter += 1
    else:
        counter += 1

Очевидно, что в данном признаке указаны не уникальные имена. Сложно понять, как этот признак может быть полезен в модели. Однако, попробуем выделить признак Kindle покупателей.

In [32]:
# Добавим колонку в список на удаление из датасета
cols_removal.append('reviewerName')

### Review Time (Unix)

Посмотрим, на распределение отзывов по датам, приведя даты к удобному виду.

In [33]:
tsmin = data.unixReviewTime.min()
ts25 = int(data.unixReviewTime.quantile(0.25))
ts50 = int(data.unixReviewTime.quantile(0.50))
ts75 = int(data.unixReviewTime.quantile(0.75))
tsmax = data.unixReviewTime.max()
print('Самый первый отзыв:', datetime.utcfromtimestamp(tsmin).strftime('%Y-%m-%d %H:%M:%S'))
print('25 квантиль:', datetime.utcfromtimestamp(ts25).strftime('%Y-%m-%d %H:%M:%S'))
print('50 квантиль:', datetime.utcfromtimestamp(ts50).strftime('%Y-%m-%d %H:%M:%S'))
print('75 квантиль:', datetime.utcfromtimestamp(ts75).strftime('%Y-%m-%d %H:%M:%S'))
print('Последний отзыв:', datetime.utcfromtimestamp(tsmax).strftime('%Y-%m-%d %H:%M:%S'))

In [34]:
# Заменим значения в столбце на категории
data['unixReviewTime'] = data['unixReviewTime'].apply(lambda x: cat_date(x))

### Votes For Review

Признак с таким количеством пропусков малоинформативен, удалим его.

In [35]:
# Добавим колонку в список на удаление из датасета
cols_removal.append('vote')

### Style

In [36]:
# Рассмотрим данные в столбце
data['style'].dropna().sample(20)

Несмотря на то,что рассматриваемый признак имеется в тестовом датасете, данные содержат очень много пропусков. Сами признаки тяжелы для обработки. Удалим столбец.

In [37]:
# Добавим колонку в список на удаление из датасета
cols_removal.append('style')

### Photos (by user or vendor)

image_x признак почти весь состоит из пропусков. Удалим столбец.

In [38]:
# Добавим колонку в список на удаление из датасета
cols_removal.append('image_x')

In [39]:
# Создадим новый признак наличия фото товара от продавца
data['vendor_photo'] = 0

counter = 0
data['image_y'] = data['image_y'].fillna(0)
for image in data['image_y']:
    if image != 0:
        data.at[counter,'vendor_photo'] = 1
        counter += 1
    else:
        counter += 1

In [40]:
# Добавим колонку в список на удаление из датасета
cols_removal.append('image_y')

In [41]:
# # # В целях эксперимента Удалим признак из рассмотрения
# columns_removal = ['vendor_photo']

### User ID, Item ID, Row ID (index)

Id всего лишь строчный индекс для каждого набора userid, itemid. Удалим признак позже. 

In [42]:
# Добавим колонку в список на удаление из датасета
cols_removal.append('Id')

Тем временем создадим новый признак популярности продукта (количество отзывов о продукте). 

In [43]:
# Создадим новый признак
item_popularity_dict = data['itemid'].value_counts().to_dict()
data['reviews_of_item'] = data['itemid'].map(item_popularity_dict)

Определим коэффициент верифицированных приобретений продукта.

In [44]:
# Создадим новый признак
verified_count = data.groupby('itemid').count()['verified']
verified_sum = data.groupby('itemid').sum()['verified']
verified_ratio = verified_sum / verified_count
verified_ratio_dict = verified_ratio.to_dict()

data['verified_purchases_ratio'] = data['itemid'].map(verified_ratio_dict)

In [45]:
# # # В целях эксперимента Удалим признаки из рассмотрения
# columns_removal.append('reviews_of_item')
# columns_removal.append('verified_purchases_ratio')

### Rating (binary) + Overall (of 5 stars)

rating - целевая переменная. Все оценки пользователей нормированы для бинарной классификации: если человек поставил оценку продукту 4 и больше, то мы считаем, что продукт ему понравился, если меньше 4, то продукт не понравился.

overall - это 5-ти бальная шкала оценок товаров пользователями. Пропусков нет. Он отсутствует в тестовом датасете. После анализа данных призак может быть удалён.

In [46]:
# Добавим колонку в список на удаление из датасета
cols_removal.append('overall')

### Review + Summary

ReviewText - более расширенная версия признака summary, его нет в тестовом датасете. Можем удалить.

In [47]:
# Преобразуем значения в колонке summary
data['summary'] = data['summary'].astype('str')
data['summary'] = data['summary'].apply(
    lambda x: re.sub("[^\w]", " ",  x.lower()))

In [48]:
# Создадим списки уникальных слов описания продукта

good_words_list = [
    "best",
    "good",
    "great",
    "love",
    "delicious",
    "nice",
    "favorite",
    "tasty",
    "perfect",
    "excellent",
    "wonderful",
    "enjoy",
    "yummy",
    "happy",
    "loves",
    "loved",
    "amazing",
    "awesome",
    "yum",
    "enjoyed",
    "fantastic",
    "perfectly",
    "wow",
    "lovely",
    "beautiful",
    "terrific",
    "enjoyable"
]

bad_words_list = [
    "awful",
    "bad",
    "disappointed",
    "unfortunate",
    "waste",
    "weird",
    "difficult",
    "terrible",
    "horrible",
    "complaint",
    "gross",
    "worst",
    "strange",
    "fake",
    "disappointing",
    "complaints",
    "poor",
    "sucks"
]

In [49]:
# Создадим новые признаки
data['good_summary'] = 0
data['bad_summary'] = 0

In [50]:
counter = 0
for summary in data['summary']:
    for word in good_words_list:
        if word in summary:
            data.at[counter,'good_summary'] = 1
            counter += 1
            break
    else:
        counter += 1
        
        
counter = 0
for summary in data['summary']:
    for word in bad_words_list:
        if word in summary:
            data.at[counter,'bad_summary'] = 1
            counter += 1
            break
    else:
        counter += 1

In [51]:
# Добавим колонки в список на удаление из датасета
cols_removal.append('reviewText')
cols_removal.append('summary')

### Description + Title

Признаки тяжелы для обработки и ненадежны. Удалим столбцы.

In [52]:
# Добавим колонки в список на удаление из датасета
cols_removal.append('description')
cols_removal.append('title')

### Category

In [53]:
# Рассмотрим данные в столбце category
data['category'].explode().value_counts()

Сложный для обработки признак, позже можем извлечь dummy-переменные и удалить столбец.

In [54]:
# Добавим колонку в список на удаление из датасета
cols_removal.append('category')

### Brand

Создадим новый признак: количество продуктов внутри бренда.

In [55]:
# Преобразуем значения в колонке brand
data['brand'] = data['brand'].astype('str')
data['brand'] = data['brand'].apply(
    lambda x: re.sub("[^\w]", "",  x.lower()))

In [56]:
# Заполним пропущенные значения
data['brand'] = data['brand'].fillna('no_brand_provided')

In [57]:
# Создадим новый признак
brand_items_dict = data.groupby('brand').count()['itemid'].to_dict()

data['items_per_brand'] = data['brand'].map(brand_items_dict)

In [58]:
# Добавим колонку в список на удаление из датасета
cols_removal.append('brand')

In [59]:
# # # В целях эксперимента Удалим признак из рассмотрения
# columns_removal.append('items_per_brand')

### Rank

In [60]:
# Рассмотрим данные в столбце
data['rank'][356911]

Признак содержит до 4 оценок товара в различных категориях. Для дальнейшего рассмотрения оставим лишь первую оценку, как описывающую "родительскую" категорию.

In [61]:
# Преобразуем значения в колонке rank
data['rank'] = data['rank'].astype(str).apply(
    lambda x: re.sub("in.*", "",  x))

data['rank'] = data['rank'].astype(str).apply(
    lambda x: re.sub("[^\d*]", "",  x))

data['rank'] = pd.to_numeric(data['rank'])

Чтобы заполнить пропуски, используем медианные значения признака rank по категориям. Но прежде вручную заполним пропуски признака в категории 'Baby'.

In [62]:
# Рассмотрим данные в столбце
data[data['main_cat'] == 'Baby']['asin'].value_counts()

Изучив уникальные коды ASIN, понимаем, что данные товары являются пищевыми продуктами. Так что изменяем категорию товаров на Grocery. 

In [63]:
baby_list = data[data['main_cat'] == 'Baby'].index.to_list()

for index in baby_list:
    data.at[index,'main_cat'] = 'Grocery'

In [64]:
# Заполним пропущенные значения
median_rank_dict = data.groupby('main_cat').median().to_dict()['rank']

data['rank'] = data['rank'].fillna(data['main_cat'].map(median_rank_dict))

In [65]:
# Заменим значения в столбце на категории
data['rank'] = data['rank'].apply(lambda x: cat_rank(x))

In [66]:
# # Добавим колонку в список на удаление из датасета
# cols_removal.append('rank')

### Also Buy + Also View

Данный признак, вероятно, говорит о том, что еще просматривали и приобретали покупатели конкретного товара. Пропуски могут говорить о том, что покупатель не смотрел больше ничего.
Создим новые признаки: факт наступления события и количество событий по каждому продукту.

In [67]:
# Создадим новые признаки
data['no_also_buy'] = pd.isna(data['also_buy']).astype('uint8')
data['no_also_view'] = pd.isna(data['also_view']).astype('uint8')

In [68]:
# Создадим новые признаки
data['also_buy_number'] = data['also_buy'].str.len()
data['also_view_number'] = data['also_view'].str.len()

data['also_buy_number'] = data['also_buy_number'].fillna(0)
data['also_view_number'] = data['also_view_number'].fillna(0)

In [69]:
# Добавим колонки в список на удаление из датасета
cols_removal.append('also_buy')
cols_removal.append('also_view')

In [70]:
# # # В целях эксперимента Удалим признаки из рассмотрения
# columns_removal.append('no_also_buy')
# columns_removal.append('no_also_view')
# columns_removal.append('also_buy_number')
# columns_removal.append('also_view_number')

### Main Category

In [71]:
# Рассмотрим данные в столбце main_cat
data['main_cat'].value_counts(dropna=False)

Указана категория, в которой определен товар. Далее сделаем из этого признака dummy-переменные или просто label encoding.

In [72]:
# # # В целях эксперимента Удалим признак из рассмотрения
# columns_removal.append('main_cat')

### Price

In [73]:
# Рассмотрим данные в столбце price
data[data['price'].str.contains("-") == True]['price'].value_counts()

In [74]:
# Преобразуем значения в колонке price
data['price'] = data['price'].astype('str').apply(
    lambda x: x.replace('$', ''))

Отдельные значения признака содержат диапазон цен, и логично будет использовать нижние значения диапазона при анализе данных.

In [75]:
# Преобразуем значения в колонке price
data['price'] = data['price'].apply(
    lambda x: re.sub("-.*", "",  x))

data['price'] = data['price'].astype('float64')

In [76]:
# Заполним пропущенные значения медианными значениями цены
median_price_dict = data.groupby('main_cat').median().transform(
    lambda x: x.fillna(x.mean())).to_dict()['price']

data['price'] = data['price'].fillna(
    data['main_cat'].map(median_price_dict))

In [77]:
# Преобразем значения в категории по цене
data['price'] = data['price'].apply(lambda x: 'low' if x < 10 else
    'high' if x > 20 else 'middle')

In [78]:
# Создадим dummy-переменные по признаку price
dummies = pd.get_dummies(data['price'], prefix = data['price'].name)
data = data.join(dummies)

In [79]:
# Добавим колонку в список на удаление из датасета
cols_removal.append('price')

In [80]:
# # # В целях эксперимента Удалим признаки из рассмотрения
# columns_removal.append('price_high')
# columns_removal.append('price_low')
# columns_removal.append('price_middle')

### Other Features (date, feature, details, similar_item, tech1, fit)

Следующие признаки содержат много пропусков, и нет понимания, как они могут быть использованы. 

In [81]:
# Добавим колонки в список на удаление из датасета
cols_removal.append('date')
cols_removal.append('feature')
cols_removal.append('details')
cols_removal.append('similar_item')
cols_removal.append('tech1')
cols_removal.append('fit')

In [82]:
# # Удаляем признаки из датасета, которые решили удалить по ходу анализа
# data.drop(cols_removal, axis=1, inplace=True)

In [83]:
# # Checking the data.
# data.info()
# data.head()

## 3.5 Детальный анализ признаков

Группировка признаков на категориальные, бинарные и числовые.

Посмотрим, какие признаки могут относиться к категориальным, бинарным, числовым.

In [84]:
# Посмотрим, на колонки, которые планируем впоследствии удалить, чтобы не включать их в анализ
print("Признаки для последующего удаления:", cols_removal)

In [85]:
# Создадим списки с разными категориями признаков
# бинарные признаки
bin_cols = ['verified', 
            'kindle_customer',
            'vendor_photo', 
            'good_summary', 
            'bad_summary', 
            'no_also_buy',
            'no_also_view', 
            'price_high', 
            'price_middle', 
            'price_low',
           ]

# категориальные переменные
cat_cols = ['unixReviewTime',
            'main_cat',
            'rank'
            ]

# числовые переменные
num_cols = ['reviews_of_item',
            'verified_purchases_ratio', 
            'also_view_number',         
            'items_per_brand', 
            'also_buy_number',
           ]

# сервисные переменные
service_cols = ['train', 'userid', 'itemid']

# целевая переменная
target_col = ['rating']

all_cols = bin_cols + cat_cols + num_cols + service_cols + target_col

print("Кол-во столбцов, для дальнейшей работы после предварительного анализа:", len(all_cols))

#### Числовые переменные: распределение, корреляционный анализ, определение значимости.

#### Распределние численных признаков.


In [86]:
# Построим распределение основных числовых признаков

print("Диаграмы распределения числовых признаков, взаимосвязь с целевой переменной")

fig, axes = plt.subplots(6, 2, figsize=(30, 40))
plt.subplots_adjust(wspace=0.5)
axes = axes.flatten()
i = 0

for col in num_cols:
    sns.distplot(data[col], ax=axes[i])
    i = i + 1
    sns.boxplot(data[col], ax=axes[i])
    i = i + 1
#     sns.scatterplot(data=data[data['train'].isna() == True],
#                     x=col, y="rating", ax=axes[i])
#     i = i + 1

In [87]:
print("Основные статистики для числовых признаков.")
display(data[num_cols].describe())

Выводы:

После построения гистограмм стало очевидно, что распределения основных числовых переменных имеют тяжёлый левый или правый хвост, влияет на разбежку между средним и медианой:
* для того, чтобы сделать распределение данных переменных более нормальным, можно работать с логарифмированными величинами этих переменных;
* поработать с выбросами;
* подумать про группировку признаков по категориям.

#### Корреляционный анализ.

Оценим корреляцию Пирсона для непрерывных переменных. Cильная корреляция между переменными вредна для линейных моделей из-за неустойчивости полученных оценок.

In [88]:
# Построим матрицу корреляций
heatmap = sns.heatmap(data[num_cols + target_col].corr(), vmin=-1,
                      vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Матрица корреляций числовых и целевой переменных')
plt.show()

Вывод:

* Не у всех числовых признаков достаточно высокая корреляция с целевой переменной.

#### Категориальные и бинарные переменные: конвертация в числовые.

Посмотрим на распределение признаков.

In [89]:
# Построим распределение основных бинарных и категориальных признаков
print("Распределение бинарных и категориальных признаков. Нажимите дважды для увелечения.")

fig, axes = plt.subplots(5, 3, figsize=(35, 35))
plt.subplots_adjust(wspace=0.5)
axes = axes.flatten()
i = 0

for col in (bin_cols + cat_cols):
    sns.histplot(data=data, x=data[col], ax=axes[i],
                 stat='count', bins=data[col].nunique())
    plt.tight_layout()
    plt.xticks(rotation=45)
    plt.title(col)
    i = i + 1

Выводы по всем графикам:

Полностью сбалансированные признаки отсутсвуют.

Особо несбалансированные признаки:

* kindle_customer - kindle покупателей в меньшинстве, но признак пока оставляем;
* bad_summary - плохие отзывы в меньшинстве, удаляем из анализа;
* main_cat - много категорий, поисследовать дополнительно и подумать над созданием новых признаков.

Сбалансированные признаки с заметно превалирующим классом:

* verified - большая часть отзывов верифицированы;
* vendor_photo - большая часть записей с фото;
* good_summary - хорошие отзывы встречаются чаще всего;
* no_also_buy - большее количество записей с отсутствием дополнительных покупок;
* no_also_view - большее количество записей с отсутствием дополнительных просмотров.

In [90]:
# Добавляем признаки в список колонок на удаление

cols_removal.append('bad_summary')
bin_cols.remove('bad_summary')

#### Преобразование бинарных переменных в числа

Чтобы алгоритмы машинного обучения могли работать с категориальными данными, их нужно преобразовать в числа. Применим LabelEncoder ко всему набору бинарных переменных.

In [91]:
# Для бинарных признаков мы будем использовать LabelEncoder
label_encoder = LabelEncoder()

for column in bin_cols:
    data[column] = label_encoder.fit_transform(data[column])

# Убедимся в преобразовании
data[bin_cols].sample(5)

## 3.6 Обработка выбросов

Признаки проанализированы, новые фичи созданы. Проведем поиск, анализ и обработку выбросов для числовых и категориальных признаков. Бинарные признаки не смотрим, т.к. мы их анализировали ранее и в них содержатся допустимые значения [0, 1].

In [92]:
# Проанализируем выбросы для категориальных и номинативных признаков
print("Отчет о наличии выбросов в датасете.\n")
for col in (num_cols + target_col):
    print("\nПризнак: ", col)
    iqr_analysis(data[col])
    print("-" * 100)

Посмотрим детальнее на признаки, где были найдены выбросы. Основная идея: сравнить мин/макс границы по test датасету и удалить лишние данные, если их объем не очень большой.

#### reviews_of_item

In [93]:
# # Посмотрим на максимальные значения признака в test
# print("Максимальное значение reviews_of_item  в test:", data[data.train == 0].reviews_of_item.max())

In [94]:
# # Посмотрим на распределение выбросов
# fig, ax = plt.subplots(figsize=(10, 5))
# sns.histplot(data=data[(data.reviews_of_item > 551)], x='reviews_of_item', hue = 'train')
# plt.title("Распределение выбросов reviews_of_item в датасете \n")
# plt.show()

Вывод:

* Выбросы по признаку представлены как в train, так и в test части. Удалять строки нельзя, т.к. необходимо строить предсказания по всем значениям reviews_of_item;

* Можно попробовать логарифмировать переменную.

Резюме: выбросы оставляем.

#### verified_purchases_ratio

In [95]:
# # Посмотрим на минимальное значения признака в test
# print("Минимальное значение verified_purchases_ratio  в test:", data[data.train == 0].verified_purchases_ratio.min())

In [96]:
# # Посмотрим на распределение выбросов
# fig, ax = plt.subplots(figsize=(10, 5))
# sns.histplot(data=data[(data.verified_purchases_ratio < 0.656)], x='verified_purchases_ratio', hue = 'train')
# plt.title("Распределение выбросов verified_purchases_ratio в датасете \n")
# plt.show()

Вывод:

* Выбросы по признаку представлены как в train, так и в test части. Удалять строки нельзя, т.к. необходимо строить предсказания по всем значениям verified_purchases_ratio;

* Можно попробовать логарифмировать переменную.

Резюме: выбросы оставляем.

#### items_per_brand

In [97]:
# # Посмотрим на максимальные значения признака в test
# print("Максимальное значение items_per_brand  в test:", data[data.train == 0].items_per_brand.max())

In [98]:
# # Посмотрим на распределение выбросов
# fig, ax = plt.subplots(figsize=(10, 5))
# sns.histplot(data=data[(data.items_per_brand > 6728)], x='items_per_brand', hue = 'train')
# plt.title("Распределение выбросов items_per_brand в датасете \n")
# plt.show()

Вывод:

* Выбросы по признаку представлены как в train, так и в test части. Удалять строки нельзя, т.к. необходимо строить предсказания по всем значениям items_per_brand;

* Можно попробовать логарифмировать переменную.

Резюме: выбросы оставляем.

#### Логарифмирование числовых признаков

Поскольку многие числовые переменные имеют смещенное распределение влево/вправо попробуем логарифмировать часть признаков.

In [99]:
cols_to_log = ['reviews_of_item',
               'verified_purchases_ratio',
               'items_per_brand',
              ]
# Применим логарифмирование ко всем числовым признакам
for col in cols_to_log:
    data[col] = data[col].apply(lambda w: np.log(w+1))

In [100]:
# # Посмотрим, как изменилось распределение
# print("Диаграмы распределения числовых признаков после логарифмирования.")
# fig, axes = plt.subplots(len(cols_to_log), figsize=(10, 15))
# axes = axes.flatten()
# i = 0

# for col in cols_to_log:
#     sns.distplot(data[col], ax=axes[i])
#     plt.title(col)
#     i = i + 1

Вывод: логарифмирование позволило привести признаки к более нормальному распределению. Оставим в таком виде.

### 3.8 Отбор признаков для моделирования

Корреляционный анализ числовых признаков

In [101]:
# Построим матрицу корреляций
# data_no_test = data[data['train']==1]
plt.figure(figsize=(10, 6))
heatmap = sns.heatmap(data[data['train'] == 1][num_cols +
                                               target_col].corr(), vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Матрица корреляций числовых и целевой переменных')
plt.show()

#### Кодирование категориальных признаков

Основная логика выбора способа кодирования:

* если у признака есть какая-то зависимость от порядкового номера категории или какая-то количественно выраженная разница между категориями, то используем label encodeing;
* если такого свойства нет и в списке большое количество значений, то пробуем one-hot-encoding.

UPD: после проведения экспериментов, было решено применить labels encoding ко всем категориальным признакам, т.к. даже после удаления статистически незначимых признаков, результаты были хуже. Требует больше времени на эксперименты.

In [102]:
# # Выведем список всех категориальных признаков и количество уникальных значений после всех оброаботок
# data[list(set(data.columns) & set(cat_cols))].nunique()

In [103]:
## Попробуем закодировать следующие признаки через label encoding, т.к. они порядковые и немногочисленные
# labels_col = ['main_cat', 'price']

In [104]:
## Попробуем закодировать следующие признаки через one hot encoding, т.к. они многочисленные и в них отсутсвует четкий порядок
# one_hot_cols = ['main_cat', 'price']

#### Label Encoding

In [105]:
# Labels encoding for all
cols_to_encode = list(set(data.columns) & set(cat_cols))
for colum in cols_to_encode:
    data[colum] = data[colum].astype('category').cat.codes

In [106]:
## Labels encoding for chosen set
#cols_to_encode_lab = labels_col
#for colum in cols_to_encode_lab:
#    data[colum] = data[colum].astype('category').cat.codes

#### One-Hot Encoding

In [107]:
## Попробуем OneHotEncoder для кодирования категориальных признаков
#cols_to_encode = one_hot_cols
#
#ohe = OneHotEncoder(sparse=False)
#
#for col in cols_to_encode:
#    df_one = pd.DataFrame(ohe.fit_transform(data[[col]]))
#     df_one.columns = ohe.get_feature_names([f'hot_{col}'])
#    data = data.drop(col, axis=1)  # удаляем колонку, которую кодировали
#    data = pd.concat([data.reset_index(drop=True), df_one.reset_index(drop=True)], axis=1)

#### Удаление признаков перед моделированием

In [108]:
# Удаляем признаки из датасета, которые решили удалить по ходу анализа
data.drop(cols_removal, axis=1, inplace=True)

In [109]:
# Смотрим, какие признаки остались
print("После обработки остались следующие признаки:", data.columns)

#### Поиск статистически значимых различий с помощью теста Стьюдента

Проверим, есть ли статистическая разница в распределении цен по категориальным и бинарным признакам, с помощью теста Стьюдента. Проверим нулевую гипотезу о том, что распределения цен по различным параметрам неразличимы.

In [110]:
# # Создадим список с колонками, которые удалим из данных ввиду их статистической незначимости
# cat_cols_remove = []

# # Проходим по колонкам, которые исключали из корреляционного анализа
# for column in (list(set(data.columns).difference(num_cols+service_cols+target_col))):
#     #print("\n\nПроверяется колонка:", column)
#     cat_cols_remove.append(get_stat_dif(column))

Вывод: присутсвуют признаки, для которых, не все значения категорий влияют на целевую переменную, такие признаки можно было бы удалить, если бы использовали one hot encoding, пока оставляем.

In [111]:
## Блок ниже использовался для случая ипользования one hot encoding, пока закомментировали.
#print(cat_cols_remove)
#
#for item in cat_cols_remove:
#    if item == None:
#        cat_cols_remove.remove(item)

In [112]:
# Оценим значимость числовых признаков
fig, ax = plt.subplots(figsize=(15, 7))
anova_df = data[data['train'] == 1].dropna().copy()
imp_num = pd.Series(f_regression(anova_df[list(set(data.columns) & set(num_cols))], anova_df['rating'])[
                    0], index=list(set(data.columns) & set(num_cols)))
imp_num.sort_values(inplace=True)
imp_num.plot(
    kind='barh', title='Значимость непрерывных переменных по ANOVA F test по всем маркам')
plt.show()

In [113]:
# Оценим значимость бинарных и категориальных признаков
fig, ax = plt.subplots(figsize=(15, 7))

anova_df = data[data['train'] == 1].dropna().copy()

# Labels encoding
cols_to_encode = list(set(anova_df.columns) & set(cat_cols))
for colum in cols_to_encode:
    anova_df[colum] = anova_df[colum].astype('category').cat.codes

imp_cat = pd.Series(mutual_info_regression(
    anova_df[list(set(data.columns) & set(bin_cols+cat_cols))], anova_df['rating'], discrete_features=True), index=list(set(data.columns) & set(bin_cols+cat_cols)))
imp_cat.sort_values(inplace=True)
imp_cat.plot(kind='barh', title='Значимость категориальных переменных')
plt.show()

Выводы:

1. Из числовых признаков очень сильное влияние имеют verified_purchases_ratio, items_per_brand, reviews_of_item;
2. Из категориальных и бинарных самое сильное влияние оказывают bad_summary, main_cat, verified.

## 4. ML DATA PREPARATION

Стандартизация числовых признаков

In [114]:
# # Стандартизация числовых переменных
# cols_to_scale = list(set(data.columns) & set(num_cols))
# data[cols_to_scale] = StandardScaler().fit_transform(data[cols_to_scale].values)

Закомментировано, т.к. стандартизация числовых признаков не оказала влияния на целевую метрику.

## 5. ML

#### Начнём с обучения "простой" модели.

In [115]:
train_data, test_data = train_test_split(train,
                                         random_state = RANDOM_STATE, 
                                         shuffle = True)

In [116]:
ratings_coo = sparse.coo_matrix((train_data['rating'].astype(int),
                                 (train_data['userid'],
                                  train_data['itemid'])))

In [117]:
model = LightFM(learning_rate = LEARNING_RATE, 
                loss = LOSS_FUNCTION,
                no_components = NUM_COMPONENTS,
                random_state = RANDOM_STATE,
                learning_schedule = LEARNING_SCHEDULE)

model = model.fit(ratings_coo, 
                  epochs = NUM_EPOCHS, 
                  num_threads = NUM_THREADS)

In [118]:
# Получаем предсказание:
preds = model.predict(test_data.userid.values,
                      test_data.itemid.values)

In [119]:
# Подсчитываем метрику roc_auc_score
sklearn.metrics.roc_auc_score(test_data.rating,preds)

In [120]:
# Cтроим ROC AUС
roc_auc_curve(test_data.rating,preds)

In [121]:
normalized_preds = (preds - preds.min())/(preds - preds.min()).max()

In [122]:
preds.min(), preds.max()

In [123]:
normalized_preds.min(), normalized_preds.max()

#### Теперь проведем обучение и получим предсказания на очищенных данных.

In [124]:
train_df = data.query('train == 1').drop('train', axis = 1)
test_df = data.query('train == 0').drop(['train', 'rating'], axis = 1)

In [125]:
train_data, test_data = train_test_split(train_df,
                                         random_state = RANDOM_STATE, 
                                         shuffle = True)

ratings_coo2 = sparse.coo_matrix((train_data['rating'].astype(int),
                                 (train_data['userid'],
                                  train_data['itemid'])))

model = LightFM(learning_rate = LEARNING_RATE, 
                loss = LOSS_FUNCTION,
                no_components = NUM_COMPONENTS,
                random_state = RANDOM_STATE,
                learning_schedule = LEARNING_SCHEDULE)

model2 = model.fit(ratings_coo2, 
                  epochs = NUM_EPOCHS, 
                  num_threads = NUM_THREADS)

In [126]:
# Получаем предсказание:
preds2 = model2.predict(test_data.userid.values,
                      test_data.itemid.values)

In [127]:
# Подсчитываем метрику roc_auc_score
sklearn.metrics.roc_auc_score(test_data.rating,preds2)

In [128]:
# Cтроим ROC AUС
roc_auc_curve(test_data.rating,preds2)

Видим, что метрика немного улучшилась!

In [129]:
normalized_preds2 = (preds2 - preds2.min())/(preds2 - preds2.min()).max()
preds2.min(), preds2.max()

In [130]:
normalized_preds2.min(), normalized_preds2.max()

In [131]:
preds_sub = model2.predict(test.userid.values,
                      test.itemid.values)

In [132]:
normalized_preds_sub = (preds_sub - preds_sub.min())/(preds_sub - preds_sub.min()).max()
preds_sub.min(), preds_sub.max()

In [133]:
normalized_preds_sub.min(), normalized_preds_sub.max()

#### Построение гибридной рекомендательной системы

In [135]:
# # Совместное использование user & item emdeddings

train_data, test_data = train_test_split(train_df, random_state = RANDOM_STATE, shuffle = True)

ratings_coo = sparse.coo_matrix((train_data['rating'].astype(int),
                                 (train_data['userid'],
                                  train_data['itemid'])))

total_users = max(train_data['userid'].max(), test_data['userid'].max()) + 1
total_items = max(train_data['itemid'].max(), test_data['itemid'].max()) + 1

# user_feature
identity_matrix = sparse.identity(total_users)
user_features = sparse.coo_matrix(identity_matrix, train_data[['verified','kindle_customer']])

#item_feature
identity_matrix = sparse.identity(total_items)
item_features = sparse.coo_matrix(identity_matrix, train_data[['main_cat','price_high','price_low','price_middle']])

model = LightFM(learning_rate = LEARNING_RATE, 
                loss = LOSS_FUNCTION,
                no_components = NUM_COMPONENTS,
                random_state = RANDOM_STATE,
                learning_schedule = LEARNING_SCHEDULE)

model = model.fit(ratings_coo, epochs=NUM_EPOCHS,
                  user_features=user_features,
                  item_features=item_features,
                  num_threads=NUM_THREADS)

preds = model.predict(test_data.userid.values,
                      test_data.itemid.values,
                      user_features=user_features,
                      item_features=item_features)

roc_auc_curve(test_data.rating, preds)

Воспользуемся обученной моделью, чтобы получить новые векторные представления книг. Формально говоря, мы просто представим каждую книгу б'ольшим количеством чисел. 

In [136]:
# Достаём эбмеддинги
item_biases, item_embeddings = model2.get_item_representations()

In [137]:
item_biases.shape, item_embeddings.shape

Мы получили эмбеддинги. Эмбеддинги нам нужны, чтобы давать предсказание к каждому товару, а точнее искать наиболее похожие. Для быстрого поиска среди большого количества товаров будем использовать метод ближайших соседей, approximate k-nn, который реализован в библиотеке nmslib.

Вместо того, чтобы перебирать все вершины, мы можем очень быстро обходить граф.

In [138]:
# Закомментируем следующие строки, т.к. в kaggle не удается импортировать библиотеку nmslib.

In [139]:
# # Создаём наш граф для поиска
# nms_idx = nmslib.init(method='hnsw', space='cosinesimil')
 
# #Начинаем добавлять наши товары в граф
# nms_idx.addDataPointBatch(item_embeddings)
# nms_idx.createIndex(print_progress=True)

In [140]:
# mapper = dict(zip(meta['asin'],meta['title']))

Попробуем написать рекомендации к какому-нибудь товару. Например, к 'cocoa'.

In [141]:
# prod_id[prod_id.title.str.find('cocoa')>=0].head(5)

Рассмотрим для примера item_id == 16104.

In [142]:
# # Ищем похожие товары
# nbm = nearest_item_nms(16104,nms_idx)[0]

In [143]:
# # Рассмотрим результаты
# prod_id[prod_id.itemid.isin(nbm)]

#### Submission

In [144]:
# Создадим submission
submission['rating'] = normalized_preds_sub

In [145]:
submission.to_csv('submission_log.csv', index=False)

## 6. SUMMARY

В ходе EDA были совмещены старые и новые данные для обучения, проведена очистка и унификация данных, обработка пустых значений, дубликатов, проведен минимально достаточный отбор признак с помощью корреляционного анализа. Были созданы новые признаки. Осталось непонятным, почему удаление дубликатов в train (~ 4.9% от общего объема) привело к ухудшению метрики на 0.03, что значительно влияет на итоговое место в leaderboard. 

Чтобы работать с моделями в библиотеке LightFm, были созданы разреженные матрицы. Данные в формате COO (координатный формат представления данных). Вместо хранения всех значений, которые включают нулевые значения, сохраняются только ненулевые значения. В COO данные представлены в виде (строка, столбец, значение).
Обучение начали с "простой" модели, которая могла бы давать рекомендации к item/user. 

Как правило не существует единой рекомендуемой метрики на все случаи жизни и каждый, кто занимается тестированием рекомендательной системы, подбирает её под свои цели. Метрика ROC-AUC наиболее подходит для оценки точности прогнозирования рекомендуемых товаров для увеличения продаж. Тем не менее бывает, что пользователи часто больше интересуются товарами в верхней части списков рекомендаций, но показатель AUC в равной степени зависит от свопов в верхней или нижней части списка рекомендаций. Это может быть недостатком, если мы, в основном, заинтересованы в поиске элементов с наивысшим рейтингом. Решить эту проблему в будущем можно с помощью LAUC (Limited Area Under the Curve). Мера LAUC может быть очень полезна для оценки рекомендательных систем, которые применяются для создания списков товаров высшего качества.

Далее мы провели обучение и предсказание на обогащенных и очищенных данных. Результат целевой метрики практически не изменился по сравнению с "простой" моделью. Следующим шагом стала работа с item_features и эмбедингами. Эмбеддинги – это векторы меньшей размерности в машинном обучении. Они используются для описания текстов, изображений, видео и много другого в поисковых и рекомендательных системах. Эмбеддинги нам нужны, чтобы давать предсказание к каждому товару, а точнее искать наиболее похожие. С помощью user/item_features появляется возможность обойти проблему холодного старта. Мы можем получить информацию о пользователе, например, при регистрации, и использовать эти данные для получения рекомендаций. Если мы используем user/item_features при обучении, то LifhtFM считает, что каждый пользователь и элемент характеризуются одной характеристикой, уникальной для этого пользователя (или элемента). Для быстрого поиска среди большого количества товаров использовали метод ближайших соседей, approximate k-nn, который реализован в библиотеке nmslib. К сожалению, данную модель мы не стали использовать для сабмита, поскольку добавление фичей результат не улучшило (таковы возможные особенности работы LightFM с матрицами item-features).

В целях оптимизации работы модели и достижения лучших результатов возможно также уделить внимание подбору гиперпараметров для LightFM, но в данном проекте эти расчеты не производились.